In [1]:
# THis is the inference script: All we need to run this is the trained model and the correct tokenizer. 
#Also some of oru custom funcitons. 

In [1]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

/home/leander/anaconda3/envs/pytorch/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.python.keras.models import load_model

In [3]:
from Clean_function import clean_note
#from tf_rec_write import load_to_memory
from Keras_Layers import Loop_Projection,Multi_Attention,sum_dum

In [4]:
#Load text and label tokenizer be crefull to select the correct one!
label_tokenizer = pickle.load(open('tokenizer/Label_Tokenizer.pkl', 'rb'))
word_tokenizer = pickle.load(open('tokenizer/Text_Tokenizer.pkl', 'rb'))

In [5]:
#reverse the Label tokenizer to get the ICD codes from the integers
reverse_label_tokenizer=dict(map(reversed, label_tokenizer.word_index.items()))

In [6]:
#load the best model 
model = load_model('best_model.keras',custom_objects={'Loop_Projection':Loop_Projection,'Multi_Attention':Multi_Attention,'sum_dum':sum_dum})

# Evaluating on Texts with Labels
You can run the following two cells to get a random example from the dataset and compare the model prediction with the true labels.

In [7]:
from mimic_data import load_dummy
import random

#get the data for an example evaluation. 
In_Text,labels=load_dummy()

In [8]:
#take a random text
example=random.randint(0,len(In_Text))
pred_text=In_Text[example]
true_labels=labels[example]

# Evaluating on a new Text
You can type a new text for prediction here, for final prediction. 

In [9]:
pred_text= '''DIMESSO CON DIASGNOSI di Anasarca. IRC 4 stadio in terapia conservativa (GFR 29 ml/min), nefropatia
    tubulointerstiziale cronica, proteinuria nefrosica secondaria (probabile paraneoplastica) in terapia steroidea.
    Infezione urinaria da Klebsiella Oxytoca ed Escherichia Coli. Cardiopatia sclerotica scompensata; FA permanente in
    terapia con EBPM, portatore di protesi vascolare aortica per aneurisma dell aorta ascendente e bioprotesi valvolare
    aortica. Diabete mellito insulino-trattato, BPCO severa.
    Uropatia ostruttiva e verosimile neoplasia vescicale infiltrante, portatore di Foley a dimora.
    Eritrodermia (Amoxicillina-Clavulanico?).Vasculopatia pluridistrettuale. Invalidamento motorio.'''

In [10]:
#clean the text, the input has to be in a list
pred_text_clean=[clean_note(pred_text)]

In [11]:
#code it correctly with the tokenizer
#important this has to be wrapped in a list again
In_code=[word_tokenizer.texts_to_sequences(pred_text_clean)]

In [17]:
#run a predict of the model 
p=model.predict(In_code)[0]

In [25]:
#get predictions larger than a threshhold. alternatively top n predictions
#top n prediction function
thresh=0.9
pred_codes=np.where(p>thresh)[1]
p_val=p[p>thresh]

In [26]:
#decode the predictions
pred_codes_decode=[reverse_label_tokenizer[j] for j in pred_codes ]

In [27]:
#predicted labels with confidence 
pd.DataFrame(np.transpose(np.vstack([pred_codes_decode,p_val]))).sort_values(by=1,ascending=False)

,0,1
1,01171,0.99997485
2,01166,0.9999739
0,01170,0.999964
